In [1]:
import numpy as np
import cv2
import os
import tensorflow as tf
import random

In [2]:
root_path = 'D:\\Coding\\ANNs\\MalariaCell\\dataset_greyscale'
classes = ['Parasitized', 'Uninfected']
folders = ['Training_set', 'Test_set']

In [3]:
labels = dict((name, index) for index, name in enumerate(classes))
labels

{'Parasitized': 0, 'Uninfected': 1}

In [4]:
Training_images = []
Training_image_labels = []
input_path = ''
for category in classes:
    input_path = os.path.join(root_path, folders[0])
    input_path = os.path.join(input_path, category)
    for image in os.listdir(input_path):
        Training_image_labels.append(labels[category])
        img_path = os.path.join(input_path, image)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = np.expand_dims(img, axis=2)
            Training_images.append(img)
        else:
            print(img)
print('DONE')

DONE


In [5]:
Testing_images = []
Testing_image_labels = []
input_path = ''
for category in classes:
    input_path = os.path.join(root_path, folders[1])
    input_path = os.path.join(input_path, category)
    for image in os.listdir(input_path):
        Testing_image_labels.append(labels[category])
        img_path = os.path.join(input_path, image)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = np.expand_dims(img, axis=2)
            Testing_images.append(img)
        else:
            print(img)
print('DONE')

DONE


In [6]:
Training_image_labels = list(zip(Training_images, Training_image_labels))
Testing_image_labels = list(zip(Testing_images, Testing_image_labels))

In [7]:
random.shuffle(Training_image_labels)
random.shuffle(Testing_image_labels)

In [8]:
Training_images, Training_image_labels = zip(*Training_image_labels)
Testing_images, Testing_image_labels = zip(*Testing_image_labels)

In [9]:
Train_count = 0
Test_count = 0

In [10]:
def next_batch(batch_size, test_set=False):
    global Train_count, Test_count
    if not test_set:
        images = np.array(Training_images[Train_count:Train_count+batch_size])
        labels = np.array(Training_image_labels[Train_count:Train_count+batch_size])
        Train_count = Train_count + batch_size
        return images, labels
    else:
        images = np.array(Testing_images[Test_count:Test_count+batch_size])
        labels = np.array(Testing_image_labels[Test_count:Test_count+batch_size])
        Test_count = Test_count+batch_size
        return images, labels

In [13]:
x = tf.placeholder(tf.float32, shape=(None, 64, 64, 1))
y_true = tf.placeholder(tf.float32)
hold_prob = tf.placeholder(tf.float32)

In [14]:
def init_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

In [15]:
def init_bias(shape):
    return tf.constant(0.1, shape=shape)

In [16]:
def conv2d(image, kernel):
    return tf.nn.conv2d(input=image, filter=kernel, strides=[1,1,1,1], padding='SAME')

In [17]:
def max_pooling2d(image):
    return tf.nn.max_pool(value=image, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [18]:
def conv_layer(image, shape):
    weights = init_weights(shape)
    bias = init_bias([shape[3]])
    return tf.nn.relu(conv2d(image, weights)+bias)

In [19]:
def dense_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    weights = init_weights([input_size, size])
    bias = init_bias([size])
    return tf.matmul(input_layer, weights)+bias

In [21]:
conv_1 = conv_layer(x, shape=[5,5,1,32])
conv_1_pool = max_pooling2d(conv_1)

In [22]:
conv_2 = conv_layer(conv_1_pool, shape=[5,5,32,64])
conv_2_pool = max_pooling2d(conv_2)

In [25]:
conv_3 = conv_layer(conv_2_pool, shape=[5,5,64,128])
conv_3_pool = max_pooling2d(conv_3)

In [26]:
flatten = tf.reshape(conv_3_pool, [-1,8*8*128])

In [27]:
dense_1 = dense_layer(flatten, size=2048)
dense_1_dropout = tf.nn.dropout(dense_1, keep_prob=hold_prob)

In [28]:
y_pred = dense_layer(dense_1_dropout, size=1)

In [29]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [30]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [31]:
init = tf.global_variables_initializer()

In [33]:
steps = 210

with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
        
        batch_x, batch_y = next_batch(100, test_set=False)
        sess.run(train, feed_dict={x:batch_x, y_true:batch_y, hold_prob:0.7})
        
        if i%100 == 0:
            print("On step : {}".format(i))
            print("Accuracy")
            matches = tf.equal(y_pred, y_true)
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            test_x, test_y = next_batch(50, test_set=True)
            print(sess.run(acc, feed_dict={x:test_x, y_true:test_y ,hold_prob:1.0}))

On step : 0
Accuracy
0.0


KeyboardInterrupt: 